There are different ways to get to the same sanswer, one of which will take 10 iterations (1 per each change in direction) and the other only takes 2.

In [2]:
import numpy
from helper import l2_norm, poisson_2d_jacobi, poisson_solution

In [3]:
# Set parameters.
nx = 101  # number of points in the x direction
ny = 101  # number of points in the y direction
xmin, xmax = 0.0, 1.0  # limits in the x direction
ymin, ymax = -0.5, 0.5  # limits in the y direction
Lx = xmax - xmin  # domain length in the x direction
Ly = ymax - ymin  # domain length in the y direction
dx = Lx / (nx - 1)  # grid spacing in the x direction
dy = Ly / (ny - 1)  # grid spacing in the y direction

# Create the gridline locations and the mesh grid.
x = numpy.linspace(xmin, xmax, num=nx)
y = numpy.linspace(ymin, ymax, num=ny)
X, Y = numpy.meshgrid(x, y)

# Create the source term.
b = (-2.0 * (numpy.pi / Lx) * (numpy.pi / Ly) *
     numpy.sin(numpy.pi * X / Lx) *
     numpy.cos(numpy.pi * Y / Ly))

# Set the initial conditions.
p0 = numpy.zeros((ny, nx))

# Compute the analytical solution.
p_exact = poisson_solution(x, y, Lx, Ly)

In [4]:
def poisson_2d_steepest_descent(p0, b, dx, dy, maxiter=20000, rtol=1e-6):
    def A(p):
        # Apply the Laplacian operator to p.
        return (-4.0 * p[1:-1, 1:-1] +
                p[1:-1, :-2] + p[1:-1, 2:] +
                p[:-2, 1:-1] + p[2:, 1:-1]) / dx**2
    p = p0.copy()
    r = numpy.zeros_like(p)  # initial residual
    Ar = numpy.zeros_like(p)  # to store the mat-vec multiplication
    conv = []  # convergence history
    diff = rtol + 1  # initial difference
    ite = 0  # iteration index
    while diff > rtol and ite < maxiter:
        pk = p.copy()
        # Compute the residual.
        r[1:-1, 1:-1] = b[1:-1, 1:-1] - A(p)
        # Compute the Laplacian of the residual.
        Ar[1:-1, 1:-1] = A(r)
        # Compute the step size.
        alpha = numpy.sum(r * r) / numpy.sum(r * Ar)
        # Update the solution.
        p = pk + alpha * r
        # Dirichlet boundary conditions are automatically enforced.
        # Compute the relative L2-norm of the difference.
        diff = l2_norm(p, pk)
        conv.append(diff)
        ite += 1
    return p, ite, conv

In [5]:
# Compute the solution using the method of steepest descent.
p, ites, conv_sd = poisson_2d_steepest_descent(p0, b, dx, dy, maxiter=20000, rtol=1e-10)
print('Method of steepest descent: {} iterations '.format(ites) +
      'to reach a relative difference of {}'.format(conv_sd[-1]))

Method of steepest descent: 2 iterations to reach a relative difference of 1.3307695446303778e-16


In [6]:
# Compute the relative L2-norm of the error.
l2_norm(p, p_exact)

8.225076220929745e-05